# Library

In [1]:
import pandas as pd
import numpy as np

## [로드] 말소 데이터

In [2]:
ex_col_df = pd.read_excel('data/220819_22.06월기준_말소_컬럼명.xlsx')
ex_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   컬럼명     42 non-null     object
dtypes: object(1)
memory usage: 464.0+ bytes


In [3]:
ex_col_df.head()

,컬럼명
0,차량관리번호
1,차량등록번호
2,본거지법정동코드
3,소유자구분
4,차종_등록정보


In [4]:
ex_col = ex_col_df['컬럼명']
len(ex_col)

42

In [5]:
# 20.8s
ex = pd.read_table('data/220819_22.06월기준_말소.txt', header=42, names=ex_col, usecols=['차대번호', '차량말소YN'], encoding='cp949')
ex.columns

Index(['차대번호', '차량말소YN'], dtype='object')

In [6]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653054 entries, 0 to 3653053
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   차대번호    object
 1   차량말소YN  object
dtypes: object(2)
memory usage: 55.7+ MB


## [로드] 컬럼명 데이터

In [7]:
col_df = pd.read_table('data/컬럼명.txt', sep=',')
col= col_df.columns

In [8]:
col

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## [로드] 파일2

In [9]:
# 2m 39.34s
df = pd.read_table('data/220819_22.06월기준_3등급.txt', encoding='cp949', header=40, names=col)
df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_14508\3108440264.py:2: DtypeWarning: Columns (14,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_3등급.txt', encoding='cp949', header=40, names=col)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8718475 entries, 0 to 8718474
Data columns (total 40 columns):
 #   Column       Dtype  
---  ------       -----  
 0   차량관리번호       object 
 1   차량등록번호       object 
 2   본거지법정동코드     int64  
 3   소유자구분        object 
 4   차종_등록정보      object 
 5   용도           object 
 6   차대번호         object 
 7   최초등록일자       int64  
 8   차량연식         int64  
 9   차량제작일자       float64
 10  검사유효일        float64
 11  제원관리번호       object 
 12  배출가스인증번호     object 
 13  배출가스등급       int64  
 14  저공해조치종류      object 
 15  저공해조치구조변경YN  object 
 16  차량말소YN       object 
 17  취득일자         float64
 18  취득금액         float64
 19  차명           object 
 20  차종_제원정보      object 
 21  차종분류         object 
 22  차종유형         object 
 23  자동차형식        object 
 24  제작사명         object 
 25  연료           object 
 26  엔진형식         object 
 27  총중량          float64
 28  적재중량         float64
 29  엔진출력         object 
 30  배기량          object 
 31  승차인원         float64
 32

In [10]:
df['차량말소YN'].unique()

array(['N'], dtype=object)

In [11]:
df2 = df.merge(ex, on='차대번호', how='left')
df2['차량말소YN_y'].unique()

array([nan, 'Y'], dtype=object)

In [12]:
df3 = df2[df2['차량말소YN_y'].isnull() == True]
df3.shape

(8718448, 41)

In [13]:
f = df3.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,7509262
5,휘발유,928635
1,LPG(액화석유가스),244449
3,경유 하이브리드,29846
0,CNG(압축천연가스),6253
4,기타연료,3


In [15]:
g = df3.groupby('연료', as_index=False)['차대번호'].groups
g['기타연료']

Int64Index([2032511, 4222289, 7797096], dtype='int64')

In [16]:
# 2AZ : 가솔린
# 1NZ : 가솔린
df3[df3['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
2032511,TOYOTA CAMRY하이브리드,2AZ
4222289,도요타 프리우스(하이브리드),1NZ
7797096,홍성캠핑트레일러,.


In [18]:
df3.loc[df3['엔진형식'] == '2AZ', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == '1NZ', '연료'] = '휘발유'
df3[df3['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
7797096,홍성캠핑트레일러,.


In [19]:
f = df3.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,7509262
5,휘발유,928637
1,LPG(액화석유가스),244449
3,경유 하이브리드,29846
0,CNG(압축천연가스),6253
4,기타연료,1


In [20]:
f.columns = ['연료', '3등급']
f

,연료,3등급
2,경유,7509262
5,휘발유,928637
1,LPG(액화석유가스),244449
3,경유 하이브리드,29846
0,CNG(압축천연가스),6253
4,기타연료,1


## [excel] 연료별 통계

In [21]:
f.to_excel('analysis/[G3]연료별_차대번호_통계.xlsx', index=False, encoding='utf-8')

C:\Users\kbj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 지역별 수량 통계

## [로드] 법정동코드

In [18]:
code = pd.read_excel('data/지역구분코드(수정).xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ad1     229 non-null    object
 1   code1   229 non-null    int64 
 2   ad2     229 non-null    object
 3   code2   229 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 7.3+ KB


In [19]:
code = code[['ad1', 'code1']].drop_duplicates(['ad1'], keep='first').reset_index(drop=True)
code

,ad1,code1
0,서울특별시,11
1,부산광역시,26
2,대구광역시,27
3,인천광역시,28
4,광주광역시,29
5,대전광역시,30
6,울산광역시,31
7,세종특별자치시,36
8,경기도,41
9,강원도,42


## 법정동 코드와 매칭

In [20]:
df['본거지법정동코드'].head()

0    4157010100
1    1165010700
2    2638010200
3    2614010100
4    1126010500
Name: 본거지법정동코드, dtype: int64

In [21]:
df['본거지법정동코드'].isnull().sum()

0

In [22]:
df['지역코드'] = df['본거지법정동코드'].astype('str').str[:2]
df['지역코드'] = df['지역코드'].astype('int')

In [23]:
df['지역코드'].head()

0    41
1    11
2    26
3    26
4    11
Name: 지역코드, dtype: int32

In [24]:
code['code1'].head()

0    11
1    26
2    27
3    28
4    29
Name: code1, dtype: int64

In [25]:
df = df.merge(code, left_on='지역코드', right_on='code1', how='left')
df[['지역코드', 'ad1']].head()

,지역코드,ad1
0,41,경기도
1,11,서울특별시
2,26,부산광역시
3,26,부산광역시
4,11,서울특별시


In [26]:
df['ad1'].isnull().sum()

0

In [27]:
l = df.groupby(['ad1'], as_index=False)['차대번호'].count()
l

,ad1,차대번호
0,강원도,349557
1,경기도,2796166
2,경상남도,839431
3,경상북도,620475
4,광주광역시,317327
5,대구광역시,547738
6,대전광역시,321244
7,부산광역시,665142
8,서울특별시,1430746
9,세종특별자치시,91006


In [30]:
l.columns = ['지역', '2등급']
l

,지역,2등급
0,강원도,349557
1,경기도,2796166
2,경상남도,839431
3,경상북도,620475
4,광주광역시,317327
5,대구광역시,547738
6,대전광역시,321244
7,부산광역시,665142
8,서울특별시,1430746
9,세종특별자치시,91006


### [excel] 지역별 수량 통계

In [31]:
l.to_excel('analysis/[G2]지역별_차대번호_통계.xlsx', index=False, encoding='utf-8')

C:\Users\kbj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 코드 마지막